In [1]:
SNLI_PATH = '/p/qdata/jm8wx/research_OLD/TextFooler_adv_results/snli_bert'

MNLI_PATH = '/p/qdata/jm8wx/research_OLD/TextFooler_adv_results/mnli_matched_bert'

In [2]:
import language_check

In [4]:
lang_tool = language_check.LanguageTool("en-US")

In [37]:
def count_grammar_errors(text):
        return len(lang_tool.check(text))

In [51]:
import pandas as pd

def df_from_tf_output(filename, join_orig_hypot=False):
    lines = open(filename).readlines()
    i = 0
    objects = []
    print(len(lines), 'lines')
    while i < len(lines):
        original_premise = lines[i].split('\t')[1].strip()
        i += 1
        original_hypothesis = lines[i].split('\t')[1].strip()
        if join_orig_hypot: # if true, orig hypothesis is a list of tokens; join w spaces
            original_hypothesis = ' '.join(eval(original_hypothesis))
        i += 1
        adv_hypothesis = lines[i].split('\t')[1].strip()
        i += 2
        obj = { 
            'original_premise': original_premise , 
            'original_hypothesis': original_hypothesis, 
            'adv_hypothesis': adv_hypothesis
         }
        objects.append( obj )
    return pd.DataFrame(objects)

In [52]:
SNLI_DF = df_from_tf_output(SNLI_PATH)
SNLI_DF.head()

3316 lines


,original_premise,original_hypothesis,adv_hypothesis
0,A person in a black and green outfit is riding...,a colorfully dressed child rides a bike,a colorfully robed child rides a bike
1,A person rolls down a hill riding a wagon as a...,A child in a wagon rolls down a hill .,A baby in a wagon rolls down a hill .
2,A man in a black tank top wearing a red plaid hat,A man wearing football pads .,A man haven calcium cloths .
3,Families with strollers waiting in front of a ...,Families have some dogs in front of a carousel,Families have some pup in front of a carousel
4,A person with dark hair in a white shirt is si...,a person is relaxing on his day off,a person is relieve on his zi off


In [34]:
import tqdm; tqdm.tqdm.pandas()

/u/jm8wx/.conda/envs/torch/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [38]:
SNLI_DF['original_hypothesis_errors'] = SNLI_DF['original_hypothesis'].progress_map(count_grammar_errors)
SNLI_DF['adv_hypothesis_errors'] = SNLI_DF['adv_hypothesis'].progress_map(count_grammar_errors)

100%|██████████| 829/829 [00:09<00:00, 86.09it/s] 


In [46]:
(SNLI_DF['adv_hypothesis_errors'] > SNLI_DF['original_hypothesis_errors']).sum() / len(SNLI_DF)

0.26417370325693607

In [53]:
MNLI_DF = df_from_tf_output(MNLI_PATH, join_orig_hypot=True)

MNLI_DF['original_hypothesis_errors'] = MNLI_DF['original_hypothesis'].progress_map(count_grammar_errors)
MNLI_DF['adv_hypothesis_errors'] = MNLI_DF['adv_hypothesis'].progress_map(count_grammar_errors)

MNLI_DF.head()

  1%|          | 7/755 [00:00<00:11, 62.98it/s]

3020 lines


100%|██████████| 755/755 [00:12<00:00, 59.15it/s]


,original_premise,original_hypothesis,adv_hypothesis,original_hypothesis_errors,adv_hypothesis_errors
0,"In Temple Bar , the bookshop at the Gallery of...",There is a bookshop at the gallery .,There is a librarians at the gallery .,1,3
1,"On Naxos , you can walk through the pretty vil...",Naxos is a place with beautiful scenery for le...,Naxos is a place with beautiful scenery for pa...,2,2
2,Expenses included in calculating net cost for ...,Net cost for education programs can be calcula...,Net cost for education programs can be calcula...,1,2
3,But the real dirty work had already been done .,There was still lots of dirty work left to be ...,There was equally lots of dirty work left to b...,1,1
4,I was pulled into the bar .,I managed to escape their grasp and ran just o...,I managed to jailbreak their grasp and ran jus...,1,1


In [54]:
(MNLI_DF['adv_hypothesis_errors'] > MNLI_DF['original_hypothesis_errors']).sum() / len(MNLI_DF)

0.21059602649006623

In [50]:
open(MNLI_PATH).readlines()[:4]

['orig premise:\tIn Temple Bar , the bookshop at the Gallery of Photography carries a large selection of photographic publications , and the Flying Pig is a secondhand bookshop .\n',
 "orig hypothesis:\t['There', 'is', 'a', 'bookshop', 'at', 'the', 'gallery', '.']\n",
 'adv hypothesis:\tThere is a librarians at the gallery .\n',
 '\n']